In [306]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import plotly.graph_objects  as go
from plotly.subplots import make_subplots

In [213]:
# Census data
features = gpd.read_file(r"C:\Users\santi\Dropbox (Via)\Remix\Data Team\Data Analytics\Intersections\data\geojson_stats_2021-03-22-20-06-31\features.geojson")
features.head(10)

,population (2019),population (2014),name,geometry
0,"1,508","1,405","Block Group 1, Tract 1","MULTIPOLYGON (((-77.06875 38.91697, -77.06741 ..."
1,"1,376","1,362","Block Group 2, Tract 1","MULTIPOLYGON (((-77.05714 38.91054, -77.05203 ..."
2,"1,108","1,245","Block Group 3, Tract 1","MULTIPOLYGON (((-77.06478 38.91040, -77.05714 ..."
3,896,"1,172","Block Group 4, Tract 1","MULTIPOLYGON (((-77.06872 38.90101, -77.06275 ..."
4,"3,922","3,844","Block Group 1, Tract 2.01","MULTIPOLYGON (((-77.07902 38.91268, -77.07310 ..."
5,"1,242","1,168","Block Group 1, Tract 2.02","MULTIPOLYGON (((-77.07310 38.91259, -77.07268 ..."
6,650,790,"Block Group 2, Tract 2.02","MULTIPOLYGON (((-77.06916 38.91069, -77.06514 ..."
7,894,915,"Block Group 3, Tract 2.02","MULTIPOLYGON (((-77.06904 38.90773, -77.06358 ..."
8,"1,923","1,678","Block Group 4, Tract 2.02","MULTIPOLYGON (((-77.07987 38.90540, -77.07831 ..."
9,"1,270","1,383","Block Group 1, Tract 3","MULTIPOLYGON (((-77.07319 38.92254, -77.07291 ..."


In [214]:
features.dtypes

population (2019)      object
population (2014)      object
name                   object
geometry             geometry
dtype: object

In [215]:
def convert_to_int(df):
    return df.apply(lambda x: x.replace(',','')).astype(int)

In [216]:
features['population (2014)'] = convert_to_int(features['population (2014)'])
features['population (2019)'] = convert_to_int(features['population (2019)'])

differences = features['population (2019)'] - features['population (2014)']

percetange_diff = differences/features['population (2014)']*100

features['delta'] = differences
features['delta_percentage'] = percetange_diff

In [217]:
# Sort the features by delta
features.sort_values(by = 'delta', ascending=False, inplace=True)
features.head()

,population (2019),population (2014),name,geometry,delta,delta_percentage
432,6909,3281,"Block Group 2, Tract 106","MULTIPOLYGON (((-77.00906 38.90251, -77.00906 ...",3628,110.576044
285,2563,898,"Block Group 2, Tract 77.03","MULTIPOLYGON (((-76.95301 38.88982, -76.94828 ...",1665,185.412027
245,4563,3109,"Block Group 1, Tract 72","MULTIPOLYGON (((-77.00911 38.88143, -77.00911 ...",1454,46.767449
339,3312,1931,"Block Group 1, Tract 88.03","MULTIPOLYGON (((-77.00173 38.90998, -76.99074 ...",1381,71.517349
334,1996,623,"Block Group 2, Tract 87.02","MULTIPOLYGON (((-77.00533 38.90921, -77.00437 ...",1373,220.385233


In [218]:
# Did the total population increase? By how much?
t_2014 = features['population (2014)'].sum()
t_2019 = features['population (2019)'].sum()

total_diff = t_2019 - t_2014
total_diff_ptg = int(total_diff/t_2014*100)

print('The total population of DC area went from {} people in 2014 to {} in 2019. This is an increase of {} people which represents a {}% change.'.format(str(t_2014), str(t_2019), str(total_diff), str(total_diff_ptg)))

The total population of DC area went from 631940 people in 2014 to 690625 in 2019. This is an increase of 58685 people which represents a 9% change.


In [219]:
# Quick view at the deltas
features['delta'].describe()

count     448.000000
mean      130.993304
std       420.050564
min     -1187.000000
25%      -103.000000
50%        98.500000
75%       308.500000
max      3628.000000
Name: delta, dtype: float64

In [220]:
# Get cut values for outliers
q1 = features['delta'].quantile(.25)
q3 = features['delta'].quantile(.75)

iqr = q3 - q1

upper_cut = q3 + 1.5*iqr
lower_cut = q1 - 1.5*iqr

In [221]:
# Quick view at the deltas in percentage
features['delta_percentage'].describe()

count    448.000000
mean            inf
std             NaN
min      -54.802831
25%       -7.782272
50%        7.500562
75%       26.427575
max             inf
Name: delta_percentage, dtype: float64

In [222]:
# Get cut values for outliers of delta percentages
q1_p = features['delta_percentage'].quantile(.25)
q3_p = features['delta_percentage'].quantile(.75)

iqr_p = q3 - q1

upper_cut_p = q3_p + 1.5*iqr_p
lower_cut_p = q1_p - 1.5*iqr_p

In [223]:
# Histogram
fig_hist = px.histogram(features, x='delta', template='simple_white', 
                        marginal="rug"
                       )
fig_hist.show()

In [224]:
# Box plot
fig_box = px.box(features, x='delta', template='simple_white', 
                 points = 'all'
                )
fig_box

In [225]:
fig_line = px.line(
    features, 
    x = 'name', y='delta', 
    template='simple_white',
    title = 'Delta'
)

fig_line.update_yaxes(showgrid=True, zeroline=True)

fig_line.show()

In [227]:
# Add the percentage to the analysis
fig = make_subplots(rows=2, cols=1, subplot_titles=('Delta', '% Delta'))

# Add absolute differences
fig.add_trace(
    go.Histogram(x = features['delta']),
    row=1, col=1
)

# Add percentage differences
fig.add_trace(
    go.Histogram(x = features['delta_percentage']),
    row=2, col=1
)

fig.update_layout(
    template='simple_white',
    xaxis = dict(visible=True, showticklabels=False),
    yaxis = dict(showgrid = True,  zeroline=True),
    yaxis2 = dict(showgrid = True,  zeroline=True),
    hovermode='x unified',

)

fig.show()

In [228]:
# Add the percentage to the analysis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('Delta', '% Delta'))

# Add absolute differences
fig.add_trace(
    go.Bar(x = features['name'], y = features['delta']),
    row=1, col=1
)

# Add percentage differences
fig.add_trace(
    go.Bar(x = features['name'], y = features['delta_percentage']),
    row=2, col=1
)

fig.update_layout(
    template='simple_white',
    xaxis = dict(visible=False, showticklabels=False),
    yaxis = dict(showgrid = True,  zeroline=True),
    yaxis2 = dict(showgrid = True,  zeroline=True),
    hovermode='x unified',
    showlegend=False

)

In [229]:
# Add the percentage to the analysis
fig = make_subplots(rows=4, cols=1, subplot_titles=('Delta','% Delta', '2014', '2019'), shared_xaxes=True,)

# Add absolute differences
fig.add_trace(
    go.Bar(x = features['name'], y = features['delta']),
    row=1, col=1
)

# Add percentage differences
fig.add_trace(
    go.Bar(x = features['name'], y = features['delta_percentage']),
    row=2, col=1
)

# Add absolute differences
fig.add_trace(
    go.Bar(y = features ['population (2014)'], x = features['name']),
    row=3, col=1
)

# Add percentage differences
fig.add_trace(
    go.Bar(y = features ['population (2019)'], x = features['name']),
    row=4, col=1
)


fig.update_layout(
    template='simple_white',
    xaxis = dict(visible=False, showticklabels=False),
#     xaxis2 = dict(visible=False, showticklabels=False),
    yaxis = dict(showgrid = True,  zeroline=True),
#     yaxis2 = dict(showgrid = True,  zeroline=True),
    hovermode='x unified',
    showlegend = False,
    height = 1000

)

In [161]:
# Add color to 

# Add the percentage to the analysis
fig = make_subplots(rows=1, cols=2, subplot_titles=('delta', 'delta_percentage'), shared_yaxes=True,)

# Add absolute differences
fig.add_trace(
    go.Bar(
        x = features ['delta'], 
        y = features['name'], 
        orientation='h',     
#         marker=dict(
#             color='rgba(246, 78, 139, 0.6)',
#             line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
#         )
    ),
    row=1, col=1
)

# Add percentage differences
fig.add_trace(
    go.Bar(x = features ['delta_percentage'], y = features['name'], orientation='h'),
    row=1, col=2
)

fig.update_layout(
    template='simple_white',
    xaxis = dict(visible=False, showticklabels=False),
    xaxis2 = dict(visible=False, showticklabels=False),
    yaxis = dict(showgrid = True,  zeroline=True),
    yaxis2 = dict(showgrid = True,  zeroline=True),
    hovermode='x unified',
    showlegend = False

)

In [162]:
fig = go.Figure()

fig.add_trace(go.Bar(name = '2014', x = features['name'], y = features ['population (2014)']))
fig.add_trace(go.Bar(name = '2019', x = features['name'], y = features ['population (2019)']))

fig.update_layout(template = 'simple_white')
fig.show()

# Maps

In [307]:
import keplergl as kp
import pydeck as pdk

In [226]:
m = kp.KeplerGl(data=dict(data=features, name=''), height=600)
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':      population (2019)  population (2014)                        name  \
432           …

In [269]:
import pydeck as pdk

x_mean = features.centroid.x.mean()
y_mean = features.centroid.y.mean()

INITIAL_VIEW_STATE = pdk.ViewState(
    latitude=y_mean, longitude=x_mean, 
    zoom=11, max_zoom=16, pitch=50, bearing=60)


geojson = pdk.Layer(
    "GeoJsonLayer",
    data = features.__geo_interface__,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation="properties.delta",
    get_fill_color="[255 * (-properties.delta_percentage), 0, properties.delta_percentage * 255]",
    get_line_color=[255, 255, 255],
)

r = pdk.Deck(layers=[geojson], initial_view_state=INITIAL_VIEW_STATE)

r.to_html("geojson_layer.html")

In [195]:
fig = px.choropleth(features, geojson=features.geometry, locations=features.index, color='delta',
                           color_continuous_scale="Viridis",
                           labels={'delta':'delta', '% delta': 'delta_percentage'},
                          )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(fitbounds="locations", visible=False)
fig.show()